In [2]:
! pip3 install pandas joblib scipy scikit-learn biopython imblearn propy3 psycopg2


Processing /home/giulia/.cache/pip/wheels/ae/51/cd/0096c52b03b32a5ffea19b70f2c56c905b5d30774229808041/psycopg2-2.9.5-cp38-cp38-linux_x86_64.whl


In [85]:
import pandas as pd
import numpy as np
import joblib
import argparse
from sklearn.ensemble import RandomForestClassifier  # random forest model
from sklearn.model_selection import GridSearchCV
from sklearn import metrics  # to calculate the accuracy of the model and confusion matrix
from sklearn.metrics import (precision_recall_curve, PrecisionRecallDisplay, auc)
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.inspection import permutation_importance
from sklearn import tree
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio import SeqIO
from imblearn.under_sampling import RandomUnderSampler
import random

# Query the training set from DB4ML

In [22]:
import psycopg2

# connect to an existing database
conn = psycopg2.connect(dbname="TEST", user="postgres", password="postgres01", host="localhost")

# open a cursor to perform database operations 
cur = conn.cursor()

# execute a command e.g. retrieve all the column of an existing table 
cur.execute('''SELECT "Test_protein".uniprot_id, 
                        "Test_protein".effector, 
                        "Test_protein_feature".feature_name, 
                        "Test_protein_feature".feature_value 
               FROM "Test_protein" 
               LEFT JOIN "Test_protein_feature" 
               ON "Test_protein".uniprot_id = "Test_protein_feature".uniprot_id;''')
db_table = cur.fetchall()

col_names = []
for el in cur.description:
    col_names.append(el[0])
df_from_db = pd.DataFrame(db_table, columns=col_names)
# close communication with the database
cur.close()
conn.close()
df_from_db

,uniprot_id,effector,feature_name,feature_value
0,A0A0A8JCG0,True,sequence length,125.00000
1,A0A0A8JCG0,True,signal peptide,0.69600
2,A0A0A8JCG0,True,transmembrane domain,1.00000
3,A0A0A8JCG0,True,aa in tr domain,20.43649
4,A0A0A8JCG0,True,first 60 aa,20.43648
...,...,...,...,...
7196,B1VAB8,False,TYR_PHOSPHO_SITE_1,0.00000
7197,B1VAB8,False,TYR_PHOSPHO_SITE_2,0.00000
7198,B1VAB8,False,AMIDATION,0.00000
7199,B1VAB8,False,EF_HAND_1,0.00000


## Format the data for TRAINING STEP

In [54]:
train_proteins = list(df_from_db["uniprot_id"].unique())
train_features = list(df_from_db["feature_name"].unique())
df_from_db_eff = df_from_db[["uniprot_id", "effector"]].groupby("uniprot_id", sort=False)
dict_reformat = {"uniprot_id":train_proteins, "name": ["effector" if el == True else "non_effector" for el in list(df_from_db_eff["effector"].first())]}

for el in train_features:
    dict_reformat[el] = []
    
for i in range(len(df_from_db)):
    dict_reformat[str(df_from_db["feature_name"].iloc[i])].append(df_from_db["feature_value"].iloc[i])

# make the reformatted table a DF
df_db_reformat = pd.DataFrame(dict_reformat)

# reformat DF columns that has to be integers
int_columns_type = ["sequence length", "transmembrane domain", "warning signal sequence", "MobiDB-lite"]
for c in list(df_db_reformat.columns):
    if c in int_columns_type:
        df_db_reformat[c] = df_db_reformat[c].astype(int)

dataset = df_db_reformat[list(df_db_reformat.columns)[1:]]
dataset

,name,sequence length,signal peptide,transmembrane domain,aa in tr domain,first 60 aa,prob N-in,warning signal sequence,MobiDB-lite,ASN_GLYCOSYLATION,CAMP_PHOSPHO_SITE,CK2_PHOSPHO_SITE,PKC_PHOSPHO_SITE,MYRISTYL,PROKAR_LIPOPROTEIN_L=0,TYR_PHOSPHO_SITE_1,TYR_PHOSPHO_SITE_2,AMIDATION,EF_HAND_1,ASN_RICH_L=0
0,effector,125,0.696,1,20.43649,20.43648,0.72986,1,0,0.285714,0.142857,0.571429,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
1,effector,125,0.696,1,20.43649,20.43648,0.72986,1,0,0.285714,0.142857,0.571429,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
2,effector,125,0.768,1,20.54200,20.54197,0.88976,1,0,0.285714,0.142857,0.571429,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
3,effector,125,0.765,1,20.54231,20.54227,0.88976,1,0,0.250000,0.125000,0.500000,0.125000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
4,effector,125,0.765,1,20.54231,20.54227,0.88976,1,0,0.250000,0.125000,0.500000,0.125000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,non_effector,351,0.124,0,0.00201,0.00000,0.02435,0,0,0.136364,0.045455,0.227273,0.318182,0.272727,0.0,0.000000,0.000000,0.000000,0.0,0.0
375,non_effector,507,0.104,0,0.01161,0.00025,0.00926,0,0,0.100000,0.166667,0.333333,0.266667,0.066667,0.0,0.000000,0.033333,0.000000,0.0,0.0
376,non_effector,377,0.219,0,0.00057,0.00000,0.00874,0,0,0.136364,0.000000,0.318182,0.227273,0.090909,0.0,0.090909,0.045455,0.045455,0.0,0.0
377,non_effector,264,0.224,0,0.00035,0.00000,0.02375,0,24,0.105263,0.105263,0.315789,0.368421,0.000000,0.0,0.052632,0.000000,0.000000,0.0,0.0


In [ ]:
# # DATASET 

# original_protein_names = pd.concat([pd.read_csv(eff_predictions, sep="\t", header=0)[["name"]], 
#                                     pd.read_csv(non_eff_predictions, sep="\t", header=0)[["name"]]], 
#                                    ignore_index=True, join="inner")

# ## EFF PREDICTIONS DF for Random Forest
# eff_pred_df = pd.read_csv(eff_predictions, sep="\t", header=0)
# eff_pred_df["name"] = ["effectors"] * len(eff_pred_df)
# # eff_pred_df = eff_pred_df.drop(columns=["ID", "organism"], axis=1)

# ## NON-EFF PREDICTIONS DF for Random Forest
# non_eff_pred_df = pd.read_csv(non_eff_predictions, sep="\t", header=0)
# non_eff_pred_df["name"] = ["non_effectors"] * len(non_eff_pred_df)

# ## COMBINE DFs (taking only effector motifs)
# for col in eff_pred_df.columns[1:]:
#     if col not in non_eff_pred_df.columns[1:]:
#         # if an effector motifs is not present in non effectors, a new column will be added to the non_effector_df
#         # having value False or 0 in this case
#         non_eff_pred_df[col] = [0] * len(non_eff_pred_df)
#     else:
#         pass


# # CONCAT AND CLEAN UP THE DATAFRAME FOR THE TRAINING
# # "inner" will take only common cols
# dataset = pd.concat([eff_pred_df, non_eff_pred_df], ignore_index=True, join="inner")
# dataset.replace([True, False], ["1", "0"], inplace=True)

# # drop the column/s having the same value for all samples == ZERO VARIANCE / ALMOST ZERO VARIANCE
# # datset = dataset.drop(columns="EF_HAND_1", axis=1)

# dataset 


# CREATE AND TRAIN/TEST THE RANDOM FOREST

In [82]:
## CREATE THE RANDOM FOREST CLASSIFIER
### STRATIFIED SHUFFLE SPLIT K-fold CROSS-VALIDATION
#### ITERATIVELY ESTIMATE THE BEST NUMBER OF DECISION TREES IN THE FOREST FOR EACH FOLD

# shuffling is useful because positive and negative elements are ordered in the dataframe and this will help to
# randomly pick the elements for each fold

input_df = pd.read_csv("/home/giulia/Workspace/PhytoPhD/period_abroad/random_forest/on_dataset20230224/clumps_features/feature_table_20230224.csv",
                       sep="\t", header=0)
common_df = input_df[list(input_df.columns)[1:]] # df without seq_id
# common_df = dataset

# skf5 = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# i = 1

# take trace of all evaluation parameters for each k-fold
kf_acc = []
kf_auc = []
kf_prec = []
kf_rec = []
kf_best_params = []
kf_n_estim = []
kf_pred_proba_test = []
kf_feat_imp = []

skf5 = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
i = 1


# SPLITTING
for train_index, test_index in skf5.split(common_df, list(common_df["name"])):
    
    x_train = common_df.iloc[train_index].loc[:, list(common_df.columns)[1:]]  # all cols except the name_col
    x_test = common_df.iloc[test_index][list(common_df.columns)[1:]]
    y_train = common_df["name"].iloc[train_index]  # only the name_col
    y_test = common_df["name"].iloc[test_index]

    # kth-fold RANDOM FOREST
    clf = RandomForestClassifier(n_estimators=100, random_state=42)  # n_estimators = number of trees in the forest
    
    ## SELECT BEST n_estimators on the TRAINING SET
    print("BEGIN best parameters-n_trees selection for Random Forest")
    params_to_test = {"n_estimators": [50, 75, 100, 1000, 5000]}
    grid_search = GridSearchCV(clf, params_to_test, n_jobs=4)
    grid_search.fit(x_train, y_train)
    print("FINISH - best parameters-n_trees selection")
    best_params = grid_search.best_params_
    clf = RandomForestClassifier(**best_params, random_state=42, max_features="sqrt")
    kf_n_estim.append(best_params["n_estimators"])
    kf_best_params.append(clf.get_params())
    print(clf.get_params())
    ## TRAIN THE MODEL USING kth-FOLD TRAINING SETS
    clf.fit(x_train, y_train)
    
       
    ## APPLY THE MODEL ON TEST SET
    y_pred = clf.predict(x_test)
    y_prob = clf.predict_proba(x_test)
    kf_pred_proba_test.append([y_prob])

    ## ACCURACY
    kth_accuracy = metrics.accuracy_score(y_test, y_pred)
    print(f"Training phase accuracy for the fold no. {i}: {kth_accuracy}")
    kf_acc.append(kth_accuracy)
    print(f"acc_train: {clf.score(x_train, y_train)}\nacc_test: {clf.score(x_test, y_test)}")

    ## F-measure
    f_score = metrics.f1_score(y_test, y_pred, average="macro")
    print(f"\nF-Measure:\t{f_score}")
    
    ### predict probability
    y_pred_prob = clf.predict_proba(x_test)

    ## PRECISION AND RECALL
    y_test_hot_encode = [0 if el == "non_effector" else 1 for el in y_test]
    y_pred_hot_encode = [0 if el == "non_effector" else 1 for el in y_pred]
    precision_perc = metrics.precision_score(y_test_hot_encode, y_pred_hot_encode, pos_label=1)
    recall_perc = metrics.recall_score(y_test_hot_encode, y_pred_hot_encode, pos_label=1)
    precision, recall, _ = precision_recall_curve(y_test_hot_encode, y_pred_hot_encode, pos_label=1)

    print(f"Precision (TP/TP+FP) on training phase: {precision_perc}")
    print(f"Recall (TP/TP+FN) on training phase: {recall_perc}")
    print(f"Precision coordinates: {precision}\nRecall(Sensitivity) coordinates: {recall}")

    kf_prec.append(precision_perc)
    kf_rec.append(recall_perc)
    
    ## Area Under the Curve
    pr_auc = auc(recall, precision)
    kf_auc.append(pr_auc)
    # disp = PrecisionRecallDisplay(precision=precision, recall=recall)
    # disp.plot()
    print(f"Area under the curve is: {pr_auc}")

    ## CONFUSION MATRIX
    conf_matrix = metrics.confusion_matrix(y_test, y_pred)
    print(f"\nconfusion matrix on k-fold test set\n{conf_matrix}")

    ## FEATURE IMPORTANCE
    ### MDI
    feat_imp = pd.Series(clf.feature_importances_, index=list(common_df.columns)[1:]).sort_values(ascending=False)
    print(f"Feature importance:\n{feat_imp}")
    kf_feat_imp.append(feat_imp.to_dict())

    
    
#     ### PERMUTATION IMPORTANCE
#     perm_imp = permutation_importance(clf, x_train, y_train, 
#                                        n_repeats=10, random_state=42, n_jobs=2, scoring='accuracy')
#     mean_perm_imp = pd.Series(perm_imp.importances_mean, index=x_train.columns).sort_values(ascending=False)
#     print(f" Permutation-based Feature Importance:\n{mean_perm_imp}")
#     for i in perm_imp.importances_mean.argsort()[::-1]:
#         print(f"{perm_imp.importances_mean[i]}\t+/- {perm_imp.importances_std[i]}")
    
    i += 1
    

print(f"Averaged accuracy for {i-1}-fold cross validation: {np.mean(kf_acc)*100}%")
print(f"Averaged precision for {i-1}-fold cross validation: {np.mean(kf_prec)*100}%")
print(f"Averaged recall for {i-1}-fold cross validation: {np.mean(kf_rec)*100}%")
print(f"Averaged best number of tree in the forest: {np.mean(kf_n_estim)}")



BEGIN best parameters-n_trees selection for Random Forest
FINISH - best parameters-n_trees selection
{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 50, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}
Training phase accuracy for the fold no. 1: 0.9583333333333334
acc_train: 1.0
acc_test: 0.9583333333333334

F-Measure:	0.9555555555555556
Precision (TP/TP+FP) on training phase: 0.9714285714285714
Recall (TP/TP+FN) on training phase: 0.918918918918919
Precision coordinates: [0.38541667 0.97142857 1.        ]
Recall(Sensitivity) coordinates: [1.         0.91891892 0.        ]
Area under the curve is: 0.9607987451737453

confusion matrix on k-fold test set
[[34  3]
 [ 1 58]]
Feature importance:
aa in tr do

KeyboardInterrupt: 

In [75]:
# OUTPUTs
# base_dir = "/home/giulia/Workspace/PhytoPhD/effectors_analysis/classification/"
# model_output_path = f"{base_dir}random_forest/rf_CLUMPs_posCLUMPs_20230213/"
model_output_path = "/home/giulia/Workspace/PhytoPhD/period_abroad/random_forest/on_dataset20230224/clumps_features/"

In [76]:
# RANDOM FOREST WITH BEST PARAMETERS TO THE ENTIRE SET OF DATA
# rf_classifier = RandomForestClassifier(n_estimators=int(np.mean(kf_best_params)), random_state=42)

# d_params = {}
# for el in clf.get_params():
#     if el == "n_estimators":
#         d_params["n_estimators"] = int(np.mean(kf_best_params))
#     else:
#         d_params[el] = clf.get_params()[el]

# the best fold is considered the one that has max accuracy 
best_fold = kf_acc.index(max(kf_acc))
# the classifier is recreated using the same parameters of the max accuracy fold
best_rf = kf_best_params[best_fold]
rf_classifier = RandomForestClassifier(**best_rf)
rf_classifier.fit(common_df[list(common_df.columns)[1:]], common_df["name"])

# SAVE THE MODEL
joblib.dump(rf_classifier, f"{model_output_path}rf_best_model.pkl")

['/home/giulia/Workspace/PhytoPhD/period_abroad/random_forest/on_dataset20230224/clumps_features/rf_best_model.pkl']

In [77]:
# PERFORMANCE ON RANDOM LABELING
acc_random = []
for i in range(100):
    random.shuffle(x := list(common_df["name"]))
    pred = rf_classifier.predict(common_df[list(common_df)[1:]])
    acc = metrics.accuracy_score(x, pred)
    acc_random.append(acc)
    i += 1

print(np.mean(acc_random))

0.5264718162839249


In [78]:
av_feat_imp = {}
for d in kf_feat_imp:
    for k,v in d.items():
        if k not in list(av_feat_imp.keys()):
            av_feat_imp[k] = [v]
        else:
            av_feat_imp[k] += [v]
av_d = {}            
for k, v in list(av_feat_imp.items()):
    av_d[k] = np.mean(v)
    
av_d_df = pd.Series(av_d, index=list(av_d.keys()))
av_d_df.to_csv(f"{model_output_path}/feature_importance.tsv", sep="\t")
print(sum(av_d_df))

1.0000000000000002


In [79]:
# print(f"Prediction probability for each fold\n{pred_proba_df}")
print(kf_pred_proba_test)

[[array([[0.98666667, 0.01333333],
       [1.        , 0.        ],
       [0.98666667, 0.01333333],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.98666667, 0.01333333],
       [0.96      , 0.04      ],
       [1.        , 0.        ],
       [0.16      , 0.84      ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.85333333, 0.14666667],
       [0.98666667, 0.01333333],
       [1.        , 0.        ],
       [0.98666667, 0.01333333],
       [1.        , 0.        ],
       [0.82666667, 0.17333333],
       [0.98666667, 0.01333333],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.01333333, 0.98666667],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [

In [80]:
# CALCUALTE PROBAILITY OF EACH CLASS PREDICTION 
y_proba = rf_classifier.predict_proba(common_df[list(common_df.columns)[1:]])
pred_proba_df = pd.DataFrame(y_proba, index=common_df["name"], columns=rf_classifier.classes_)
pred_proba_df["seq_id"] = list(input_df["seq_id"])
print(pred_proba_df)
pred_proba_df.to_csv(f"{model_output_path}/rf_best_model_predictions_probability.tsv", sep="\t", index=False)


              effector  non_effector                  seq_id
name                                                        
effector          1.00          0.00  tr|B3R0L2|B3R0L2_PHYMT
effector          1.00          0.00  tr|C0H5W6|C0H5W6_ONYPE
effector          1.00          0.00  tr|Q2NJQ2|Q2NJQ2_AYWBP
effector          1.00          0.00  tr|Q2NJA6|Q2NJA6_AYWBP
effector          1.00          0.00  tr|Q2NK94|Q2NK94_AYWBP
...                ...           ...                     ...
non_effector      0.02          0.98     sp|Q6YRK0|RF1_ONYPE
non_effector      0.00          1.00    sp|Q6YRL3|DNAA_ONYPE
non_effector      0.04          0.96    sp|Q8G8D1|REPP_ONYPE
non_effector      0.00          1.00    sp|Q9KJU0|GRPE_PEWBP
non_effector      0.00          1.00    sp|B1VAB8|HRCA_PHYAS

[479 rows x 3 columns]


In [ ]:
# PERMUTATION-BASED FEATURE IMPORTANCE

per_importance = permutation_importance(clf, common_df[list(common_df.columns)[1:]], common_df["name"], 
                                       n_repeats=30, random_state=42, scoring='accuracy')
for i in per_importance.importances_mean.argsort()[::-1]:
    print(f"{per_importance.importances_mean[i]}\t+/- {per_importance.importances_std[i]}")
# imp_importance = pd.Series(clf.feature_importances_, index=list(common_df.columns)[1:]).sort_values(ascending=False)
imp_importance = rf_classifier.feature_importances_
print(imp_importance)

In [ ]:
# OVERFIT THE MODEL TO SEE IF THE PERMUTATION FEATURE IMPORTANCE CHANGE 
over_dataset = dataset[['name', 'sequence length', 'signal peptide', 'transmembrane domain',
       'Phob signal peptide', 'Phob transmembrane domain', 'h_bin1', 'h_bin2', 'h_bin3']]
over_dataset


# 4-FOLD CROSS VALIDATION RF

In [83]:
## CREATE THE RANDOM FOREST CLASSIFIER
### STRATIFIED SHUFFLE SPLIT K-fold CROSS-VALIDATION
#### ITERATIVELY ESTIMATE THE BEST NUMBER OF DECISION TREES IN THE FOREST FOR EACH FOLD

# shuffling is useful because positive and negative elements are ordered in the dataframe and this will help to
# randomly pick the elements for each fold

input_df = pd.read_csv("/home/giulia/Workspace/PhytoPhD/period_abroad/random_forest/on_dataset20230224/clumps_features/feature_table_20230224.csv",
                       sep="\t", header=0)
common_df = input_df[list(input_df.columns)[1:]] # df without seq_id
# common_df = dataset

# take trace of all evaluation parameters for each k-fold
kf_acc = []
kf_auc = []
kf_prec = []
kf_rec = []
kf_best_params = []
kf_n_estim = []
kf_pred_proba_test = []
kf_feat_imp = []

skf4 = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
i = 1


# SPLITTING
for train_index, test_index in skf4.split(common_df, list(common_df["name"])):
    
    x_train = common_df.iloc[train_index].loc[:, list(common_df.columns)[1:]]  # all cols except the name_col
    x_test = common_df.iloc[test_index][list(common_df.columns)[1:]]
    y_train = common_df["name"].iloc[train_index]  # only the name_col
    y_test = common_df["name"].iloc[test_index]

    # kth-fold RANDOM FOREST
    clf = RandomForestClassifier(n_estimators=100, random_state=42)  # n_estimators = number of trees in the forest
    
    ## SELECT BEST n_estimators on the TRAINING SET
    print("BEGIN best parameters-n_trees selection for Random Forest")
    params_to_test = {"n_estimators": [50, 75, 100, 1000, 5000]}
    grid_search = GridSearchCV(clf, params_to_test, n_jobs=4)
    grid_search.fit(x_train, y_train)
    print("FINISH - best parameters-n_trees selection")
    best_params = grid_search.best_params_
    clf = RandomForestClassifier(**best_params, random_state=42, max_features="sqrt")
    kf_n_estim.append(best_params["n_estimators"])
    kf_best_params.append(clf.get_params())
    print(clf.get_params())
    ## TRAIN THE MODEL USING kth-FOLD TRAINING SETS
    clf.fit(x_train, y_train)
    
       
    ## APPLY THE MODEL ON TEST SET
    y_pred = clf.predict(x_test)
    y_prob = clf.predict_proba(x_test)
    kf_pred_proba_test.append([y_prob])

    ## ACCURACY
    kth_accuracy = metrics.accuracy_score(y_test, y_pred)
    print(f"Training phase accuracy for the fold no. {i}: {kth_accuracy}")
    kf_acc.append(kth_accuracy)
    print(f"acc_train: {clf.score(x_train, y_train)}\nacc_test: {clf.score(x_test, y_test)}")

    ## F-measure
    f_score = metrics.f1_score(y_test, y_pred, average="macro")
    print(f"\nF-Measure:\t{f_score}")
    
    ### predict probability
    y_pred_prob = clf.predict_proba(x_test)

    ## PRECISION AND RECALL
    y_test_hot_encode = [0 if el == "non_effector" else 1 for el in y_test]
    y_pred_hot_encode = [0 if el == "non_effector" else 1 for el in y_pred]
    precision_perc = metrics.precision_score(y_test_hot_encode, y_pred_hot_encode, pos_label=1)
    recall_perc = metrics.recall_score(y_test_hot_encode, y_pred_hot_encode, pos_label=1)
    precision, recall, _ = precision_recall_curve(y_test_hot_encode, y_pred_hot_encode, pos_label=1)

    print(f"Precision (TP/TP+FP) on training phase: {precision_perc}")
    print(f"Recall (TP/TP+FN) on training phase: {recall_perc}")
    print(f"Precision coordinates: {precision}\nRecall(Sensitivity) coordinates: {recall}")

    kf_prec.append(precision_perc)
    kf_rec.append(recall_perc)
    
    ## Area Under the Curve
    pr_auc = auc(recall, precision)
    kf_auc.append(pr_auc)
    # disp = PrecisionRecallDisplay(precision=precision, recall=recall)
    # disp.plot()
    print(f"Area under the curve is: {pr_auc}")

    ## CONFUSION MATRIX
    conf_matrix = metrics.confusion_matrix(y_test, y_pred)
    print(f"\nconfusion matrix on k-fold test set\n{conf_matrix}")

    ## FEATURE IMPORTANCE
    ### MDI
    feat_imp = pd.Series(clf.feature_importances_, index=list(common_df.columns)[1:]).sort_values(ascending=False)
    print(f"Feature importance:\n{feat_imp}")
    kf_feat_imp.append(feat_imp.to_dict())

    
    i += 1
    

print(f"Averaged accuracy for {i-1}-fold cross validation: {np.mean(kf_acc)*100}%")
print(f"Averaged precision for {i-1}-fold cross validation: {np.mean(kf_prec)*100}%")
print(f"Averaged recall for {i-1}-fold cross validation: {np.mean(kf_rec)*100}%")
print(f"Averaged best number of tree in the forest: {np.mean(kf_n_estim)}")

BEGIN best parameters-n_trees selection for Random Forest
FINISH - best parameters-n_trees selection
{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}
Training phase accuracy for the fold no. 1: 0.9666666666666667
acc_train: 1.0
acc_test: 0.9666666666666667

F-Measure:	0.9644444444444444
Precision (TP/TP+FP) on training phase: 0.9772727272727273
Recall (TP/TP+FN) on training phase: 0.9347826086956522
Precision coordinates: [0.38333333 0.97727273 1.        ]
Recall(Sensitivity) coordinates: [1.         0.93478261 0.        ]
Area under the curve is: 0.9685276679841898

confusion matrix on k-fold test set
[[43  3]
 [ 1 73]]
Feature importance:
transmemb

# 4-fold Repeated stratified kfold

In [86]:
## CREATE THE RANDOM FOREST CLASSIFIER
### STRATIFIED SHUFFLE SPLIT K-fold CROSS-VALIDATION
#### ITERATIVELY ESTIMATE THE BEST NUMBER OF DECISION TREES IN THE FOREST FOR EACH FOLD

# shuffling is useful because positive and negative elements are ordered in the dataframe and this will help to
# randomly pick the elements for each fold

input_df = pd.read_csv("/home/giulia/Workspace/PhytoPhD/period_abroad/random_forest/on_dataset20230224/clumps_features/feature_table_20230224.csv",
                       sep="\t", header=0)
common_df = input_df[list(input_df.columns)[1:]] # df without seq_id
# common_df = dataset

# take trace of all evaluation parameters for each k-fold
kf_acc = []
kf_auc = []
kf_prec = []
kf_rec = []
kf_best_params = []
kf_n_estim = []
kf_pred_proba_test = []
kf_feat_imp = []

skf4 = RepeatedStratifiedKFold(n_splits=4, n_repeats=5, random_state=42)
i = 1


# SPLITTING
for train_index, test_index in skf4.split(common_df, list(common_df["name"])):
    
    x_train = common_df.iloc[train_index].loc[:, list(common_df.columns)[1:]]  # all cols except the name_col
    x_test = common_df.iloc[test_index][list(common_df.columns)[1:]]
    y_train = common_df["name"].iloc[train_index]  # only the name_col
    y_test = common_df["name"].iloc[test_index]

    # kth-fold RANDOM FOREST
    clf = RandomForestClassifier(n_estimators=100, random_state=42)  # n_estimators = number of trees in the forest
    
    ## SELECT BEST n_estimators on the TRAINING SET
    print("BEGIN best parameters-n_trees selection for Random Forest")
    params_to_test = {"n_estimators": [50, 75, 100, 1000, 5000]}
    grid_search = GridSearchCV(clf, params_to_test, n_jobs=4)
    grid_search.fit(x_train, y_train)
    print("FINISH - best parameters-n_trees selection")
    best_params = grid_search.best_params_
    clf = RandomForestClassifier(**best_params, random_state=42, max_features="sqrt")
    kf_n_estim.append(best_params["n_estimators"])
    kf_best_params.append(clf.get_params())
    print(clf.get_params())
    ## TRAIN THE MODEL USING kth-FOLD TRAINING SETS
    clf.fit(x_train, y_train)
    
       
    ## APPLY THE MODEL ON TEST SET
    y_pred = clf.predict(x_test)
    y_prob = clf.predict_proba(x_test)
    kf_pred_proba_test.append([y_prob])

    ## ACCURACY
    kth_accuracy = metrics.accuracy_score(y_test, y_pred)
    print(f"Training phase accuracy for the fold no. {i}: {kth_accuracy}")
    kf_acc.append(kth_accuracy)
    print(f"acc_train: {clf.score(x_train, y_train)}\nacc_test: {clf.score(x_test, y_test)}")

    ## F-measure
    f_score = metrics.f1_score(y_test, y_pred, average="macro")
    print(f"\nF-Measure:\t{f_score}")
    
    ### predict probability
    y_pred_prob = clf.predict_proba(x_test)

    ## PRECISION AND RECALL
    y_test_hot_encode = [0 if el == "non_effector" else 1 for el in y_test]
    y_pred_hot_encode = [0 if el == "non_effector" else 1 for el in y_pred]
    precision_perc = metrics.precision_score(y_test_hot_encode, y_pred_hot_encode, pos_label=1)
    recall_perc = metrics.recall_score(y_test_hot_encode, y_pred_hot_encode, pos_label=1)
    precision, recall, _ = precision_recall_curve(y_test_hot_encode, y_pred_hot_encode, pos_label=1)

    print(f"Precision (TP/TP+FP) on training phase: {precision_perc}")
    print(f"Recall (TP/TP+FN) on training phase: {recall_perc}")
    print(f"Precision coordinates: {precision}\nRecall(Sensitivity) coordinates: {recall}")

    kf_prec.append(precision_perc)
    kf_rec.append(recall_perc)
    
    ## Area Under the Curve
    pr_auc = auc(recall, precision)
    kf_auc.append(pr_auc)
    # disp = PrecisionRecallDisplay(precision=precision, recall=recall)
    # disp.plot()
    print(f"Area under the curve is: {pr_auc}")

    ## CONFUSION MATRIX
    conf_matrix = metrics.confusion_matrix(y_test, y_pred)
    print(f"\nconfusion matrix on k-fold test set\n{conf_matrix}")

    ## FEATURE IMPORTANCE
    ### MDI
    feat_imp = pd.Series(clf.feature_importances_, index=list(common_df.columns)[1:]).sort_values(ascending=False)
    print(f"Feature importance:\n{feat_imp}")
    kf_feat_imp.append(feat_imp.to_dict())

    
    i += 1
    

print(f"Averaged accuracy for {i-1}-fold cross validation: {np.mean(kf_acc)*100}%")
print(f"Averaged precision for {i-1}-fold cross validation: {np.mean(kf_prec)*100}%")
print(f"Averaged recall for {i-1}-fold cross validation: {np.mean(kf_rec)*100}%")
print(f"Averaged best number of tree in the forest: {np.mean(kf_n_estim)}")

BEGIN best parameters-n_trees selection for Random Forest
FINISH - best parameters-n_trees selection
{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}
Training phase accuracy for the fold no. 1: 0.9666666666666667
acc_train: 1.0
acc_test: 0.9666666666666667

F-Measure:	0.9644444444444444
Precision (TP/TP+FP) on training phase: 0.9772727272727273
Recall (TP/TP+FN) on training phase: 0.9347826086956522
Precision coordinates: [0.38333333 0.97727273 1.        ]
Recall(Sensitivity) coordinates: [1.         0.93478261 0.        ]
Area under the curve is: 0.9685276679841898

confusion matrix on k-fold test set
[[43  3]
 [ 1 73]]
Feature importance:
transmemb

FINISH - best parameters-n_trees selection
{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 50, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}
Training phase accuracy for the fold no. 6: 0.975
acc_train: 1.0
acc_test: 0.975

F-Measure:	0.9736668861092825
Precision (TP/TP+FP) on training phase: 0.9574468085106383
Recall (TP/TP+FN) on training phase: 0.9782608695652174
Precision coordinates: [0.38333333 0.95744681 1.        ]
Recall(Sensitivity) coordinates: [1.         0.97826087 0.        ]
Area under the curve is: 0.9720205057045945

confusion matrix on k-fold test set
[[45  1]
 [ 2 72]]
Feature importance:
transmembrane domain       0.261995
first 60 aa                0.231868
aa in tr domain       

FINISH - best parameters-n_trees selection
{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 1000, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}
Training phase accuracy for the fold no. 12: 1.0
acc_train: 1.0
acc_test: 1.0

F-Measure:	1.0
Precision (TP/TP+FP) on training phase: 1.0
Recall (TP/TP+FN) on training phase: 1.0
Precision coordinates: [0.38655462 1.         1.        ]
Recall(Sensitivity) coordinates: [1. 1. 0.]
Area under the curve is: 1.0

confusion matrix on k-fold test set
[[46  0]
 [ 0 73]]
Feature importance:
first 60 aa                0.232968
aa in tr domain            0.191819
warning signal sequence    0.177815
transmembrane domain       0.166686
signal peptide             0.083086

FINISH - best parameters-n_trees selection
{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 75, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}
Training phase accuracy for the fold no. 18: 0.9666666666666667
acc_train: 1.0
acc_test: 0.9666666666666667

F-Measure:	0.9644444444444444
Precision (TP/TP+FP) on training phase: 0.9772727272727273
Recall (TP/TP+FN) on training phase: 0.9347826086956522
Precision coordinates: [0.38333333 0.97727273 1.        ]
Recall(Sensitivity) coordinates: [1.         0.93478261 0.        ]
Area under the curve is: 0.9685276679841898

confusion matrix on k-fold test set
[[43  3]
 [ 1 73]]
Feature importance:
transmembrane domain       0.284388
first 60 aa                0.23